In [242]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score

# Importing and simple cleaning of data

First of all, some simple cleaning. We remove features that have nothing to do with the skin color. We remove playerShort (we will use 'player' later for aggregating), birthday, Alpha_3 (since it is the same as refCountry) and photoID.

There are missing values for height, weight and position. And also 163 dyads miss the information for the implicit association test and the explicit bias scores.

First we will remove the refrees that got in the data by mistake (as mentioned in the preprocessing article) and see if the problem is solved then. After removing these refrees, there were still 110 samples with missing data which we just removed.

The missing values of height and weight are replaced by the respective means and the missing values in position are replaced by the most frequently occuring position.

In [243]:
df = pd.read_csv ('CrowdstormingDataJuly1st.csv')
df.head ()

,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,victories,...,rater2,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
0,lucas-wilchez,Lucas Wilchez,Real Zaragoza,Spain,31.08.1983,177.0,72.0,Attacking Midfielder,1,0,...,0.50,1,1,GRC,0.326391,712.0,0.000564,0.396000,750.0,0.002696
1,john-utaka,John Utaka,Montpellier HSC,France,08.01.1982,179.0,82.0,Right Winger,1,0,...,0.75,2,2,ZMB,0.203375,40.0,0.010875,-0.204082,49.0,0.061504
2,abdon-prats,Abdón Prats,RCD Mallorca,Spain,17.12.1992,181.0,79.0,NaN,1,0,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
3,pablo-mari,Pablo Marí,RCD Mallorca,Spain,31.08.1993,191.0,87.0,Center Back,1,1,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
4,ruben-pena,Rubén Peña,Real Valladolid,Spain,18.07.1991,172.0,70.0,Right Midfielder,1,1,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002


In [244]:
# remove values without skin color rating
df_train_raw = df [pd.notnull (df ['rater1'])]

In [245]:
# drop unimportant features
df_train_raw = df_train_raw.drop(['playerShort', 'birthday', 'Alpha_3', 'photoID'], axis=1)

In [246]:
# remove refrees that are not supposed to be in here
refrees_to_remove = df_train_raw.groupby('refNum').sum()[df_train_raw.groupby('refNum').sum()['games'] < 22].index.tolist()
df_train_raw = df_train_raw[~df_train_raw.refNum.isin(refrees_to_remove)]

In [247]:
# remove samples that don't have IAT or Exp score information
df_train_raw = df_train_raw[df_train_raw.meanIAT.notnull()]

In [248]:
# set missing height, weight and position values to mean
mean_height = df_train_raw.height.mean()
mean_weight = df_train_raw.weight.mean()
most_frequent_position = df_train_raw['position'].value_counts().index[0]

df_train_raw.loc[df_train_raw.height.isnull(),'height'] = mean_height
df_train_raw.loc[df_train_raw.weight.isnull(),'weight'] = mean_weight
df_train_raw['position'] = df_train_raw['position'].fillna(most_frequent_position)

# Advanced pre-processing

We create a new feature called "skinColor" which is the average of both ratings.

Position, club and leagueCountry are not numerical, we use dummy variables to make them numerical. For now we left out the club feature because it would cause a lot of dummy variables and we assume it is not a significant feature. We will check in the end if we get significantly better result if we add this feature.

**TODO: Aggregrate per player**

**TODO: feature engineering**

In [256]:
# create feature "skinColor"
raters = pd.DataFrame (df_train_raw [['rater1', 'rater2']].mean (axis=1))
raters.columns = ['skinColor']
df_train = pd.merge (raters, df_train_raw.drop (['rater1', 'rater2'], 1), right_index = True, left_index = True)

In [257]:
# add dummy variables for position, club and leagueCountry
n_positions = len(df_train.position.unique())
n_club = len(df_train.club.unique())
n_leagueCountry = len(df_train.leagueCountry.unique())

d_positions = pd.get_dummies(df_train['position'])
d_club = pd.get_dummies(df_train['club'])
d_leagueCountry = pd.get_dummies(df_train['leagueCountry'])

df_train = pd.concat([df_train, d_positions, d_club, d_leagueCountry], axis=1)
df_train = df_train.drop(['position', 'club', 'leagueCountry', 'player'], 1)

# Making the model

In [254]:
clf = RandomForestClassifier(n_estimators=10)
Y = np.asarray(df_train['skinColor'], dtype="int")
X = df_train

# Cross validation

In [255]:
cross_val_score(clf, X, Y, cv=5, scoring='accuracy')

array([ 1.,  1.,  1.,  1.,  1.])